In [ ]:
# ! pip install -qU \
#     openai \
#     pinecone-client \
#     langchain \
#     tiktoken \
#     dotenv

In [1]:
from dotenv import dotenv_values

config = dotenv_values("../.env")

## Initializing Embedding Model and Vector DB

In [2]:
from FlagEmbedding import FlagModel

model = FlagModel('BAAI/bge-large-en-v1.5', 
                query_instruction_for_retrieval="Represent this sentence for searching relevant passages: ",
                use_fp16=True)

embed = lambda x: model.encode(x).tolist()            

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
PINECONE_API = config['PINECONE_API']
PINECONE_ENV = config['PINECONE_ENV']

In [4]:
import pinecone

# find API key in console at app.pinecone.io
YOUR_API_KEY = PINECONE_API
# find ENV (cloud region) next to API key in console
YOUR_ENV = PINECONE_ENV

index_name = 'llm-recommender-system'
pinecone.init(
    api_key=YOUR_API_KEY,
    environment=YOUR_ENV
)

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='dotproduct',
        dimension=1536  # 1536 dim of text-embedding-ada-002
    )

In [5]:
index = pinecone.Index(index_name)
index_stats_response = index.describe_index_stats()

print(index_stats_response)

{'dimension': 1024,
 'index_fullness': 0.01199,
 'namespaces': {'': {'vector_count': 1199}},
 'total_vector_count': 1199}


## Indexing Embedding Vectors

In [6]:
import pandas as pd

In [7]:
sample_books = pd.read_csv("../data/sample_books.csv")

In [10]:
sample_books

,Id,Name,Description
0,1374307,"Tough Times Never Last, but Tough People Do!","Name your problem, and you name your possibili..."
1,1334737,Black Gold,This bracing novel introduces vivid African-Am...
2,1598771,The Autobiography of Lincoln Steffens,Rediscover a man Americans turned to not only ...
3,2179832,Bright Shiny Morning,One of the most celebrated and controversial a...
4,4421731,The Socially Skilled Child Molester: Different...,Know what signs indicate a child molester!<br ...
...,...,...,...
95,2442839,"National Portrait Gallery, London",-- The first full catalogue of this major art ...
96,2787050,Collecting Toy Cars and Trucks: A Collector's ...,This indispensable identification and value gu...
97,4506874,Self Related Cognitions In Anxiety And Motivation,First published in 1986. Routledge is an impri...
98,3002398,Predators: A Pop-up Book with Revolutionary Te...,Get up close and personal with some of the wor...


In [11]:
sample_books['category'] =  sample_books.apply(lambda x: 'popular' if x.name < 50 else 'recommended', axis=1)

In [12]:
sample_books['user_id'] = sample_books.apply(lambda x: x.name % 10 + 1 if x['category'] == 'recommended' else None, axis = 1)

In [13]:
sample_books['ranking'] = sample_books.groupby('user_id').cumcount() + 1

In [14]:
sample_books.rename(columns={'Id' : 'id', 'Name': 'title', 'Description': 'description'}, inplace=True)

In [94]:
data = sample_books

In [96]:
data[data['user_id'] == 1]

,id,title,description,category,user_id,ranking
50,4589735,Informal Economy: A Research G,First published in 1991. Routledge is an impri...,recommended,1.0,1.0
60,1112628,In Amma's Healing Room: Gender and Vernacular ...,"""[I]t is extremely salubrious to see the ways ...",recommended,1.0,2.0
70,3041182,Vision & Prophecy in Amos Revised,In 1955 John D. W. Watts presented the faculty...,recommended,1.0,3.0
80,2243440,Environmental Regulation: Statutory Supplement...,This excellent resource takes full advantage o...,recommended,1.0,4.0
90,4502407,Hooded Americanism: The History of the Ku Klux...,"""The only work that treats Ku Kluxism for the ...",recommended,1.0,5.0


In [105]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_size = 100

texts = []
metadatas = []

for i in tqdm(range(0, len(data), batch_size)):
    # get end of batch
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    # first get metadata fields for this record
    metadatas = [
    {
        'category': record['category'],
        'title': record['title'],
        'description': record['description'],
        **({'user_id': str(int(record['user_id']))} if record['category'] == 'recommended' else {})
    }
    for _, record in batch.iterrows()]
    # get the list of contexts / documents
    documents = batch['description']
    # create document embeddings
    embeds = embed.embed_documents(documents)
    # get IDs
    ids = batch['id'].astype(str)
    # add everything to pinecone
    index.upsert(vectors=zip(ids, embeds, metadatas))

100%|██████████| 1/1 [00:04<00:00,  4.98s/it]


In [106]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.001,
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100}

## Creating Vector Store and Querying

In [6]:
from langchain.schema.vectorstore import VectorStoreRetriever
from langchain.callbacks.manager import (
        AsyncCallbackManagerForRetrieverRun,
        CallbackManagerForRetrieverRun,
    )
from typing import (
    TYPE_CHECKING,
    Any,
    Callable,
    ClassVar,
    Collection,
    Dict,
    Iterable,
    List,
    Optional,
    Tuple,
    Type,
    TypeVar,
)
from langchain.schema.document import Document

class PineconeVectorStoreRetriever(VectorStoreRetriever):

     def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        if self.search_type == "similarity":
            docs = self.vectorstore.similarity_search(query, **self.search_kwargs)
        elif self.search_type == "similarity_score_threshold":
            docs_and_similarities = (
                self.vectorstore.similarity_search_with_relevance_scores(
                    query, **self.search_kwargs
                )
            )
            for doc, score in docs_and_similarities:
                doc.metadata = {**doc.metadata, **{"score": 1-score}}
            docs = [doc for doc, _ in docs_and_similarities]
        elif self.search_type == "mmr":
            docs = self.vectorstore.max_marginal_relevance_search(
                query, **self.search_kwargs
            )
        else:
            raise ValueError(f"search_type of {self.search_type} not allowed.")
        return docs 

In [7]:
from langchain.vectorstores import Pinecone

class PineconeModified(Pinecone):
    
    def as_retriever(self, **kwargs: Any) -> VectorStoreRetriever:
        """Return VectorStoreRetriever initialized from this VectorStore.

        Args:
            search_type (Optional[str]): Defines the type of search that
                the Retriever should perform.
                Can be "similarity" (default), "mmr", or
                "similarity_score_threshold".
            search_kwargs (Optional[Dict]): Keyword arguments to pass to the
                search function. Can include things like:
                    k: Amount of documents to return (Default: 4)
                    score_threshold: Minimum relevance threshold
                        for similarity_score_threshold
                    fetch_k: Amount of documents to pass to MMR algorithm (Default: 20)
                    lambda_mult: Diversity of results returned by MMR;
                        1 for minimum diversity and 0 for maximum. (Default: 0.5)
                    filter: Filter by document metadata

        Returns:
            VectorStoreRetriever: Retriever class for VectorStore.

        Examples:

        .. code-block:: python

            # Retrieve more documents with higher diversity
            # Useful if your dataset has many similar documents
            docsearch.as_retriever(
                search_type="mmr",
                search_kwargs={'k': 6, 'lambda_mult': 0.25}
            )

            # Fetch more documents for the MMR algorithm to consider
            # But only return the top 5
            docsearch.as_retriever(
                search_type="mmr",
                search_kwargs={'k': 5, 'fetch_k': 50}
            )

            # Only retrieve documents that have a relevance score
            # Above a certain threshold
            docsearch.as_retriever(
                search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.8}
            )

            # Only get the single most similar document from the dataset
            docsearch.as_retriever(search_kwargs={'k': 1})

            # Use a filter to only retrieve documents from a specific paper
            docsearch.as_retriever(
                search_kwargs={'filter': {'paper_title':'GPT-4 Technical Report'}}
            )
        """
        tags = kwargs.pop("tags", None) or []
        tags.extend(self._get_retriever_tags())

        return PineconeVectorStoreRetriever(vectorstore=self, **kwargs, tags=tags)

In [8]:
text_field = "description"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = PineconeModified(
    index, embed, text_field
)
     

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [9]:
query = "adventure story"

vectorstore.similarity_search_with_score(
    query, 
    k=3,
    filter={'category': 'popular'}
)



[(Document(page_content='Born the runt of his litter and gambled away to a rusty old river man, the Newfoundland pup Seaman doesn’t imagine his life will be marked by any kind of glory. But when he meets Captain Meriwether Lewis, Seaman finds himself on a path that will make history. Lewis is setting off on his landmark search for the Northwest Passage, and he takes Seaman along.\xa0Sharing the curiosity and spirit of his new master, the intrepid dog proves himself a valuable companion at every turn. \xa0\xa0\xa0\xa0 Part history, part science—and all adventure—this is the thrilling tale of America’s greatest journey of discovery.<br /><b>\n  <i>\xa0\xa0\xa0\xa0 Includes an author’s note and a reader<b><i>’</i></b>s guide.</i>\n</b>', metadata={'category': 'popular', 'title': "The Captain's Dog: My Journey with the Lewis and Clark Tribe"}),
  0.69451189),
 (Document(page_content='You and Carlos are on an expedition to the world\'s highest mountains in the remote kingdom of Nepal to sea

In [18]:
query = "history"

vectorstore.similarity_search_with_score(
    query, 
    k=3,
    filter={'user_id' : '1' , 'category': 'recommended'}
)
     

[]

## Tools definition


In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.schema.vectorstore import VectorStore
from langchain.chains.query_constructor.ir import StructuredQuery, Visitor
from langchain.schema.language_model import BaseLanguageModel
from langchain.retrievers.self_query.pinecone import PineconeTranslator
from langchain.chains.query_constructor.base import load_query_constructor_runnable

from typing import Any, Dict, List, Optional, Sequence, Tuple, Type, Union

In [16]:
def _get_builtin_translator(vectorstore: VectorStore) -> Visitor:
    """Get the translator class corresponding to the vector store class."""
    BUILTIN_TRANSLATORS: Dict[Type[VectorStore], Type[Visitor]] = {
        PineconeModified: PineconeTranslator,
    }
    
    if vectorstore.__class__ in BUILTIN_TRANSLATORS:
        return BUILTIN_TRANSLATORS[vectorstore.__class__]()
    else:
        raise ValueError(
            f"Self query retriever with Vector Store type {vectorstore.__class__}"
            f" not supported."
        )

class PineconeSelfQueryRetriever(SelfQueryRetriever):

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        vectorstore: VectorStore,
        document_contents: str,
        metadata_field_info: Sequence[Union[AttributeInfo, dict]],
        structured_query_translator: Optional[Visitor] = None,
        chain_kwargs: Optional[Dict] = None,
        enable_limit: bool = False,
        use_original_query: bool = False,
        **kwargs: Any,
    ) -> "SelfQueryRetriever":
        if structured_query_translator is None:
            structured_query_translator = _get_builtin_translator(vectorstore)
        chain_kwargs = chain_kwargs or {}

        if "allowed_comparators" not in chain_kwargs:
            chain_kwargs[
                "allowed_comparators"
            ] = structured_query_translator.allowed_comparators
        if "allowed_operators" not in chain_kwargs:
            chain_kwargs[
                "allowed_operators"
            ] = structured_query_translator.allowed_operators
        query_constructor = load_query_constructor_runnable(
            llm,
            document_contents,
            metadata_field_info,
            enable_limit=enable_limit,
            **chain_kwargs,
        )
        return cls(
            query_constructor=query_constructor,
            vectorstore=vectorstore,
            use_original_query=use_original_query,
            structured_query_translator=structured_query_translator,
            **kwargs,
        )

In [12]:
# chat completion llm

llm = ChatOpenAI(
    model_name='gpt-4-1106-preview',
    temperature=0.0
)

In [13]:
# conversational memory

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [14]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

### Tool 1 - Generic Recommendation

In [115]:
metadata_field_info=[
    AttributeInfo(
        name="user_id",
        description="The user ID of the book recommendation",
        type="string",
    ),
    AttributeInfo(
        name="category",
        description="The type of entry (popular or recommended)",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="title",
        description="The title of the book",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="The description of the book",
        type="float"
    )
]
document_content_description = "The description of the book"

pineconeSelfQueryRetriever = PineconeSelfQueryRetriever.from_llm(llm, vectorstore, document_content_description, metadata_field_info, verbose=True)

In [25]:
## USER ID IS SPECIFICALLY DETERMINED IN FRONTEND

In [ ]:
# retrieval qa chain

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=pineconeSelfQueryRetriever
)

In [27]:
%%time

qa.run("I am user 1 and recommend some books for me")

CPU times: user 229 ms, sys: 37.8 ms, total: 267 ms
Wall time: 31.3 s


'As user 1, I don\'t have specific information about your preferences, interests, or reading history. However, I can recommend a variety of books across different genres that are popular or critically acclaimed. Here are some suggestions:\n\n1. Fiction:\n   - "The Night Circus" by Erin Morgenstern – A magical competition between two young illusionists set in a mysterious circus.\n   - "Where the Crawdads Sing" by Delia Owens – A coming-of-age murder mystery set in the North Carolina marshes.\n\n2. Non-Fiction:\n   - "Educated" by Tara Westover – A memoir about a woman who grows up in a strict and abusive household in rural Idaho and her journey to earning a PhD from Cambridge University.\n   - "Sapiens: A Brief History of Humankind" by Yuval Noah Harari – An exploration of the history of humans from the Stone Age to the 21st century.\n\n3. Science Fiction/Fantasy:\n   - "Dune" by Frank Herbert – A classic science fiction novel about politics, religion, and ecology on the desert planet 

#### Optimized version

In [ ]:
def format_docs_title_source(docs):
    res = {
        'result' : "\n\n".join([f"{i+1}. {d.metadata['title']} : {d.page_content}" for i,d in enumerate(docs)]),
        'source_documents' : [d.metadata['book_id'] for d in docs]
    }
    return str(res)

In [119]:
pinecone_retriever = vectorstore.as_retriever(
                search_kwargs={'k' : 5, 
                                'filter': {'user_id' : '1147','category': 'recommended'}})

generic_qa = pinecone_retriever | format_docs_title_source

eval(generic_qa.invoke("Recommend me books about history"))

{'result': "1. The Map Thief : Beijing, China, 1421: It is a momentous time for the Ming Dynasty. Honoring the completion of the Forbidden City, a fleet of unprecedented size sets sail under Admiral Zheng He. Zheng’s mission is to chart the globe, trading for riches and bringing glory to China’s emperor. Among the crew is the talented cartographer and navigator Ma Zhi, whose work will lead to the first true map of the world–but whose accomplishment will vanish when the fleet returns to a very different China than the one it left.<br /><br />Lisbon, Portugal, 1496: At the height of Portugal’s maritime domination during the Age of Discovery, the legendary explorer Vasco da Gama embarks on a quest to find a sea route to India. On board is navigator Antonio Coehlo, who guards Portugal’s most secret treasure: a map that already shows the way.<br /><br />New York, present day: Mara Coyne’s new client has left her uneasy. Republican kingmaker Richard Tobias has hired her, he says, because of 

### Tool 2 - Popular Recommendation

In [21]:
# retrieval qa chain

popular_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(
        search_kwargs={'k' : 5, 
                       'filter': {'category': 'recommended'}}),
    return_source_documents = True
)

In [22]:
def run_qa_chain(question):
    return str(popular_qa(question))

In [23]:
a = run_qa_chain("Recommend me books about history")

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [24]:
eval(a)

{'query': 'Recommend me books about history',
 'result': 'Conquistador by Buddy Levy is a book about history that you might find interesting, as it covers the last days of the Aztec empire and the clash between Hernán Cortés and Montezuma.',
 'source_documents': [Document(page_content='In an astonishing work of scholarship that reads like an adventure thriller, historian Buddy Levy records the last days of the Aztec empire and the two men at the center of an epic clash of cultures. <br /><b>\n  <br />\n</b><i>“I and my companions suffer from a disease of the heart which can be cured only with gold.”</i> —<i>Hernán Cortés<br /><br /></i>It was a moment unique in human history, the face-to-face meeting between two men from civilizations a world apart. Only one would survive the encounter. In 1519, Hernán Cortés arrived on the shores of Mexico with a roughshod crew of adventurers and the intent to expand the Spanish empire. Along the way, this brash and roguish conquistador schemed to con

#### Optimized version

In [51]:
def format_docs_title_source(docs):
    res = {
        'result' : "\n\n".join([f"{i+1}. {d.metadata['title']} : {d.page_content}" for i,d in enumerate(docs)]),
        'source_documents' : [d.metadata['book_id'] for d in docs]
    }
    return str(res)

In [52]:
pinecone_retriever = vectorstore.as_retriever(
                search_kwargs={'k' : 5, 
                                'filter': {'category': 'recommended'}})

popular_qa = pinecone_retriever | format_docs_title_source

eval(popular_qa.invoke("Recommend me books about history"))

{'result': '1. Conquistador: Hernán Cortés, King Montezuma, and the Last Stand of the Aztecs : In an astonishing work of scholarship that reads like an adventure thriller, historian Buddy Levy records the last days of the Aztec empire and the two men at the center of an epic clash of cultures. <br /><b>\n  <br />\n</b><i>“I and my companions suffer from a disease of the heart which can be cured only with gold.”</i> —<i>Hernán Cortés<br /><br /></i>It was a moment unique in human history, the face-to-face meeting between two men from civilizations a world apart. Only one would survive the encounter. In 1519, Hernán Cortés arrived on the shores of Mexico with a roughshod crew of adventurers and the intent to expand the Spanish empire. Along the way, this brash and roguish conquistador schemed to convert the native inhabitants to Catholicism and carry off a fortune in gold. That he saw nothing paradoxical in his intentions is one of the most remarkable—and tragic—aspects of this unforgett

### Tool 3 - Specific Recommendation

In [35]:
recommended_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(
        search_kwargs={'k' : 5, 
                       'filter': {'user_id' : '1', 'category': 'recommended'}}),
    return_source_documents = True
)

In [ ]:
recommended_qa.invoke("Recommend me books about history")

#### Optimized Version

In [84]:
def format_docs_title(docs):
    return "\n\n".join([f"{i+1}. {d.metadata['title']} : {d.page_content}" for i,d in enumerate(docs)])

In [85]:
pineconeRetreiver = vectorstore.as_retriever(
        search_kwargs={'k' : 5, 
                       'filter': {'user_id' : '1147', 'category': 'recommended'}})

recommended_qa = pineconeRetreiver | format_docs_title

recommended_qa.invoke("fantasy books")

"1. Servants of the Map : Ranging across two centuries, and from the western Himalaya to an Adirondack village, these wonderfully imagined stories and novellas travel the territories of yearning and awakening, of loss and unexpected discovery. A mapper of the highest mountain peaks realizes his true obsession. A young woman afire with scientific curiosity must come to terms with a romantic fantasy. Brothers and sisters, torn apart at an early age, are beset by dreams of reunion. Throughout, Barrett's most characteristic theme—the happenings in that borderland between science and desire—unfolds in the diverse lives of unforgettable human beings. Although each richly layered tale stands independently, readers of <em>Ship Fever</em> (National Book Award winner) and Barrett's extraordinary novel <em>The Voyage of the Narwhal</em>, will discover subtle links both among these new stories and to characters in the earlier works.\n\n2. The Map Thief : Beijing, China, 1421: It is a momentous tim

In [86]:
def format_docs_title_source(docs):
    res = {
        'result' : "\n\n".join([f"{i+1}. {d.metadata['title']} : {d.page_content}" for i,d in enumerate(docs)]),
        'source_documents' : [d.metadata['book_id'] for d in docs]
    }
    return str(res)

In [87]:
recommended_qa_with_source =  pineconeRetreiver | format_docs_title_source

In [88]:
recommended_qa_with_source.invoke('fantasy books')

'{\'result\': "1. Servants of the Map : Ranging across two centuries, and from the western Himalaya to an Adirondack village, these wonderfully imagined stories and novellas travel the territories of yearning and awakening, of loss and unexpected discovery. A mapper of the highest mountain peaks realizes his true obsession. A young woman afire with scientific curiosity must come to terms with a romantic fantasy. Brothers and sisters, torn apart at an early age, are beset by dreams of reunion. Throughout, Barrett\'s most characteristic theme—the happenings in that borderland between science and desire—unfolds in the diverse lives of unforgettable human beings. Although each richly layered tale stands independently, readers of <em>Ship Fever</em> (National Book Award winner) and Barrett\'s extraordinary novel <em>The Voyage of the Narwhal</em>, will discover subtle links both among these new stories and to characters in the earlier works.\\n\\n2. The Map Thief : Beijing, China, 1421: It 

In [89]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from operator import itemgetter

# Check if the recommended books is match the user's query
chain = (
    {"recommended_books": recommended_qa, "query": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(
        """
        Check if the document recommends a book. Say "yes" or "no".

        Recommended Books: 
        {recommended_books}

        Classification:"""
    )
    | llm
    | StrOutputParser()
)

# If yes
recommended_chain = (
    {"recommended_books" : RunnablePassthrough()}
    | PromptTemplate.from_template(
        """
        You are an expert in recommended books. \
        Give the user book recommendation books using below information. \
        Always start with "I have some books recommendation that is tailored to your taste. \
        
        Recommended Books: 
        {recommended_books}
        """
    )
    | llm
)

# If no
popular_chain = (
    {"recommended_books": popular_qa, "question": RunnablePassthrough()}
    | PromptTemplate.from_template(
        """
        You are an expert in recommended books. \
        Give the user book recommendation books using below information. \
        Always start with "I have some popular books that I can recommend for you. \
        
        Recommended Books: 
        {recommended_books}
        """
    )
    | llm
)

from langchain.schema.runnable import RunnableBranch

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

full_chain = (
    {
        "topic": (
            {"recommended_books": recommended_qa , "query": RunnablePassthrough()}
            | ChatPromptTemplate.from_template(
                """
                Check if the document recommends a book. Say "yes" or "no".

                Recommended Books: 
                {recommended_books}

                Classification:"""
            )
            | llm
            | StrOutputParser()
            ), 
        "query": RunnablePassthrough()
    }
    | RunnableBranch(
        (lambda x: "yes" in x["topic"].lower() or "Yes" in x["topic"].lower(), (lambda x :  x['query']) | recommended_qa_with_source),
        (lambda x: "no" in x["topic"].lower() or "No" in x["topic"].lower(), (lambda x :  x['query']) | popular_chain),
        (lambda x :  x['query']) | popular_chain
        )
    | StrOutputParser()
)


In [91]:
full_chain.invoke("Recommend some books about romance for me")

'{\'result\': \'1. Servants of the Map : Ranging across two centuries, and from the western Himalaya to an Adirondack village, these wonderfully imagined stories and novellas travel the territories of yearning and awakening, of loss and unexpected discovery. A mapper of the highest mountain peaks realizes his true obsession. A young woman afire with scientific curiosity must come to terms with a romantic fantasy. Brothers and sisters, torn apart at an early age, are beset by dreams of reunion. Throughout, Barrett\\\'s most characteristic theme—the happenings in that borderland between science and desire—unfolds in the diverse lives of unforgettable human beings. Although each richly layered tale stands independently, readers of <em>Ship Fever</em> (National Book Award winner) and Barrett\\\'s extraordinary novel <em>The Voyage of the Narwhal</em>, will discover subtle links both among these new stories and to characters in the earlier works.\\n\\n2. The Blue Star : Jim Glass has fallen

In [58]:
async for chunk in full_chain.astream_log(
    "I am a user 1 and recommend some books about romance for me"
):
    print("-" * 80)
    print(chunk)

--------------------------------------------------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '55334475-78e4-4d0d-baeb-e3df16f4d4fa',
            'logs': {},
            'streamed_output': []}})
--------------------------------------------------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableParallel',
  'value': {'end_time': None,
            'final_output': None,
            'id': '560ccf1d-811d-439e-935c-031e0e9805f6',
            'metadata': {},
            'name': 'RunnableParallel',
            'start_time': '2023-11-13T01:00:20.986',
            'streamed_output_str': [],
            'tags': ['seq:step:1'],
            'type': 'chain'}})
--------------------------------------------------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableSequence',
  'value': {'end_time': None,
            'final_output': None,
            'id': '045b

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:371: UserWarning: The aapply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


--------------------------------------------------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI/final_output',
  'value': {'generations': [[{'generation_info': {'finish_reason': 'stop'},
                              'message': AIMessage(content='This document does not answer the question\nScore: 0'),
                              'text': 'This document does not answer the '
                                      'question\n'
                                      'Score: 0',
                              'type': 'ChatGeneration'}]],
            'llm_output': {'model_name': 'gpt-4-1106-preview',
                           'token_usage': {'completion_tokens': 12,
                                           'prompt_tokens': 338,
                                           'total_tokens': 350}},
            'run': None}},
 {'op': 'add',
  'path': '/logs/ChatOpenAI/end_time',
  'value': '2023-11-13T01:00:23.804'})
-----------------------------------------

In [37]:
full_chain.invoke("I am user 1 and recommend me books about history")

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'I have some book recommendations that are tailored to your taste. Since you\'re interested in history, "Hooded Americanism" is a book that might interest you as it covers the history of the Ku Klux Klan over a century, providing detailed insights from primary materials.'

In [59]:
full_chain.invoke("history")

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'I have some popular books that I can recommend for you:\n\n1. For a deep dive into medieval warfare and its impact on English and French history, you might be interested in a book that provides the "First large-scale study of Crécy and its context, bringing out its full significance in English and French history." This book promises to offer a comprehensive analysis of the Battle of Crécy and its historical implications, making it a valuable read for enthusiasts of military history and medieval studies.\n\n2. If you\'re fascinated by ancient civilizations and their cultural legacies, consider a book that is "a timely work which seeks to place Egypt in its true historical context." This book explores the African origins of the ancient Egyptians, their advancements in arts and sciences, and their social systems. It\'s an enlightening read that challenges conventional narratives and is recommended for those interested in Egyptology and African history.\n\n3. For those intrigued by the co

In [58]:
full_chain.invoke("history books")

'1. Hooded Americanism: The History of the Ku Klux Klan, 3rd ed. : "The only work that treats Ku Kluxism for the entire period of it\'s existence. . . . <i>the </i>authoritative work on the period. <i>Hooded Americanism</i> is exhaustive in its rich detail and its use of primary materials to paint the picture of a century of terror. It is comprehensive, since it treats the entire period, and enjoys the perspective that the long view provides. It is timely, since it emphasizes the undeniable persistence of terrorism in American life."—John Hope Franklin\n\n2. Informal Economy: A Research G : First published in 1991. Routledge is an imprint of Taylor &amp; Francis, an informa company.\n\n3. Environmental Regulation: Statutory Supplement and Guide to Internet Resources, 2002 Edition : This excellent resource takes full advantage of on-line information and students\' familiarity with it. Reflecting the author\'s dual focus on environmental regulations and the policy considerations that sha

In [148]:
ai_message = llm.invoke('Good')

In [151]:
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()

def update_env(book_ids: list):
    # print book_ids to a file
    with open('book_ids.txt', 'w') as f:
        f.write(str(book_ids))

def feedback(message):
    def parse_feedback(message):
        chain = (
                {'user_feedback' : RunnablePassthrough()}
                | PromptTemplate.from_template(
                    """
                    You are a parser. Based on the user's feedback output the book id of the book that the user likes. \
                    
                    User Feedback:
                    {user_feedback}

                    Your response should be a list of comma separated values, eg: `foo, bar, baz`
                    """
                )
                | llm
                | output_parser
        )
        return chain.invoke(message)

    # hrsnya ada thread yg ngeupdate env
    book_ids = parse_feedback(message)
    update_env(book_ids)


    ai_message = llm.invoke(message)
    return ai_message.content

### Tool Consolidation

In [152]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Generic Recommendation',
        func=generic_qa.invoke,
        description=(
            'use this tool when the user asking for book recommendation without any specific preference (not popular) just input "book" as the query parameter'
        ),
        return_direct = True
    ),
    Tool(
        name='Specific Recommendation',
        func=full_chain.invoke,
        description=(
            'use this tool when the user asking for book recommendation with a specific preference (genre, theme, etc.)'
        ),
        return_direct = True
    ),
    Tool(
        name='Popular Recommendation',
        func=popular_qa.invoke,
        description=(
            'use this tool when the user asking for popular book recommendation without any specific preference'
        ),
        return_direct = True
    ),
    Tool(
        name='Generic Prompt',
        func=feedback,
        description=(
            'use this tool when the user give a feedback to the recommendation and input the user input as the query parameter'
        ),
        return_direct = True
    ),
]

## Conversation Agent

In [153]:
# conversational memory

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [154]:
from langchain.agents import (initialize_agent, AgentType)

agent = initialize_agent(
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    # max_iterations=3,
    # early_stopping_method='generate',
    memory=conversational_memory,
    return_source_documents=True
)

In [155]:
def agent_invoke(message):
    res = agent.invoke(message)

    output = eval(res['output'])

    user_output = "Here are some book recommendations for you: \n" + output['result'] + "\n\n Which one do you want to read first?"
    book_ids = output['source_documents']

    return user_output, book_ids

In [156]:
user_output, book_ids = agent_invoke('Recommend me popular books about romance')



> Entering new AgentExecutor chain...
Thought: The user is asking for popular book recommendations within the romance genre. I will use the Popular Recommendation tool with "romance" as the input to find suitable books.

Action:
```
{
  "action": "Popular Recommendation",
  "action_input": "romance"
}
```


Observation: {'result': '1. A Hero of Our Time : The first example of the psychological novel in Russia, <b>A Hero of Our Time</b> influenced Tolstoy, Dostoyevsky, and Chekhov, and other great nineteenth-century masters that followed. Its hero, Pechorin, is Byronic in his wasted gifts, his cynicism, and his desire for any kind of action-good or ill-that will stave off boredom. Outraging many critics when it was first published in 1840, <b>A Hero of Our Time</b> follows Pechorin as he embarks on an exciting adventure involving brigands, smugglers, soldiers, rivals, and lovers.<br /><br />This edition includes a new introduction, chronology, suggestions for further reading, maps, an

In [103]:
user_output, book_ids = agent_invoke('Recommend me books about fantasy')



> Entering new AgentExecutor chain...
Thought: The user is asking for book recommendations within the fantasy genre but hasn't specified any particular sub-genre or theme they are interested in. I will use the Specific Recommendation tool to provide a list of fantasy books.

Action:
```
{
  "action": "Specific Recommendation",
  "action_input": {
    "genre": "fantasy"
  }
}
```


Observation: {'result': "1. The Sound of Thunder : <i>The Sound of Thunder</i>, a Courtney Family Adventure by Wilbur Smith<br /><br />The year is 1899. The British Empire is at the height of its power. But in South Africa, proud Dutch colonists defy the Queen and her army--and a lush, wild world is torn apart by guns, spears, and swords.<br /><br />Sean Courtney had been tragically separated from his family, but the Boer War has brought him back to his homeland--and into the sights of his enemy's guns. It has also returned him to Ruth Friedman, the only woman Sean can love, even though he shouldn't. As Sea

In [133]:
user_output, book_ids = agent_invoke('Recommend me any books')



> Entering new AgentExecutor chain...
Thought: The user is asking for a book recommendation without specifying any particular genre or theme. I will use the Generic Recommendation tool to provide a suggestion.

Action:
```
{
  "action": "Generic Recommendation",
  "action_input": "book"
}
```


Observation: {'result': "1. The Vagrants : Brilliant and illuminating, this astonishing debut novel by the award-winning writer Yiyun Li is set in China in the late 1970s, when Beijing was rocked by the Democratic Wall Movement, an anti-Communist groundswell designed to move China beyond the dark shadow of the Cultural Revolution toward a more enlightened and open society. In this powerful and beautiful story, we follow a group of people in a small town during this dramatic and harrowing time, the era that was a forebear of the Tiananmen Square uprising.<br /><br />Morning dawns on the provincial city of Muddy River. A young woman, Gu Shan, a bold spirit and a follower of Chairman Mao, has ren

## Feedback Loop

In [157]:
agent.invoke('I like book number 1')



> Entering new AgentExecutor chain...
Thought: The user has given feedback indicating they like "book number 1," but without context, it's unclear what book they're referring to. I'll use the Generic Prompt tool to ask for clarification.

Action:
```
{
  "action": "Generic Prompt",
  "action_input": "I like book number 1"
}
```


Observation: I'm glad to hear that you enjoyed the first book in a series or a standalone book labeled as "number 1." If you're looking for recommendations on what to read next, whether it's the next book in the series or something similar in genre or style, feel free to provide more details about the book you liked, and I can help guide you to your next read. If you're looking for more information about the book or its author, I can help with that as well. Just let me know what you're interested in!


> Finished chain.


{'input': 'I like book number 1',
 'chat_history': [HumanMessage(content='Recommend me popular books about romance'),
  AIMessage(content="{'result': '1. A Hero of Our Time : The first example of the psychological novel in Russia, <b>A Hero of Our Time</b> influenced Tolstoy, Dostoyevsky, and Chekhov, and other great nineteenth-century masters that followed. Its hero, Pechorin, is Byronic in his wasted gifts, his cynicism, and his desire for any kind of action-good or ill-that will stave off boredom. Outraging many critics when it was first published in 1840, <b>A Hero of Our Time</b> follows Pechorin as he embarks on an exciting adventure involving brigands, smugglers, soldiers, rivals, and lovers.<br /><br />This edition includes a new introduction, chronology, suggestions for further reading, maps, and full explanatory notes.<br />\\n\\n2. A Hero of Our Time : The first example of the psychological novel in Russia, <b>A Hero of Our Time</b> influenced Tolstoy, Dostoyevsky, and Chekh